In [ ]:
import numpy as np

The Wasserstein barycenter problem attempts to summarize a collection $(\mu_i){_{i=1,...,n}}$ of probability distributions by taking their weighted average with respect to the Wasserstein distance. Following [Agueh and Carlier 2011], given a set of weights $\alpha = (\alpha_i){_{i=1,...,n}}$, it is defined as the following problem : 
\begin{equation}
\min_{\mu} \sum_{i=1}^{k} \alpha_i W_2^2(\mu, \mu_i)
\end{equation}

In [15]:
from PIL import Image 
#charger une image 

def preprocess_image(image_path, new_size = (256, 256)):
    image = Image.open(image_path).convert('L')
    image = image.resize(new_size)
    image = image / np.sum(image)  # Normalisation pour que la somme des pixels = 1
    
    return image

In [130]:
def wasserstein_barycenter(mu, a,  Kernel, n_iter = 2000, alpha = None):
    #v_i is in R^N
    #v is in R^K*N with k the number of distributions
    #mu is the array of distribution mu_1,..., mu_K and its shape is N*K
    K = mu.shape[1]
    N = mu.shape[0]
    
    if alpha == None:
        alpha = np.ones(K)/K #poids uniformes 
    w = np.ones((N,K))
    v = np.copy(w)
    d = np.copy(w)
    epsilon = 1e-19

    for i in range(n_iter):
        
        # First step of the Bergman projection (onto C1)
        target_mu = np.ones(N)
        for i in range(K):
            w[:,i] = mu[:,i]/(Kernel(a*v[:,i]) + epsilon)
            #d[:,i] = v[:,i] * Kernel(a*w[:,i])
            target_mu = target_mu * (v[:,i] * Kernel(a*w[:,i]))**alpha[i]
                
        # Second projection (onto C2)
        for i in range(K):
            v[:,i] =  v[:,i]*target_mu / (Kernel(a*w[:,i])+epsilon)
            
    return target_mu



In [134]:
from scipy.ndimage import gaussian_filter
entropy = 5
sigma = 1
f_heatkernel = lambda x : gaussian_filter(x, sigma = sigma/2)
N = 256
dirac1 = np.zeros(N)
dirac2 = np.zeros(N)
dirac1[20]= 1
dirac2[150]= 1
a = np.ones(N)/N


barycenter_mu = wasserstein_barycenter(np.array([dirac1,dirac2]).T, a, f_heatkernel,alpha=[1/2,1/2],n_iter=500)
